<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [2]:
import numpy as np

In [3]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [4]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [5]:
def get_corpus_lists(corpus_input):
    term_list = list()
    unique_term_list = list()
    for document in corpus_input:
        terms = list(document.split(' '))
        term_list.append(terms)
        for term in terms:
            if term not in unique_term_list:
                unique_term_list.append(term)
    return term_list, unique_term_list

term_list, unique_term_list = get_corpus_lists(corpus)

print(term_list)
print(unique_term_list)


[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]
['que', 'dia', 'es', 'hoy', 'martes', 'el', 'de', 'muchas', 'gracias']


### 2- OneHot encoding
Dada una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [6]:
def text_list_to_OHE(input_document_list, input_unique_term_list):
    return_array = np.zeros([len(input_document_list), len(input_unique_term_list)], dtype=np.uint8)
    for index_document, document in enumerate(input_document_list):
        for term in document:
            try:
                unique_index = input_unique_term_list.index(term)
            except ValueError:
                unique_index = -1
            if unique_index != -1:
               return_array[index_document][unique_index] = 1

    return return_array

text_list_to_OHE(term_list, unique_term_list)

array([[1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 1]], dtype=uint8)

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [7]:
def text_list_to_freq(input_document_list, input_unique_term_list):
    return_array = np.zeros([len(input_document_list), len(input_unique_term_list)], dtype=np.uint8)
    for index_document, document in enumerate(input_document_list):
        for term in document:
            try:
                unique_index = input_unique_term_list.index(term)
            except ValueError:
                unique_index = -1
            if unique_index != -1:
               return_array[index_document][unique_index] = return_array[index_document][unique_index] + 1

    return return_array

text_list_to_freq(term_list, unique_term_list)

array([[1, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 2, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 1]], dtype=uint8)

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [8]:
def text_list_to_TF_IDF(input_document_list, input_unique_term_list):
    #return_array = np.zeros([1, len(input_unique_term_list)], dtype=np.uint8)
    OHE_array = text_list_to_OHE(input_document_list, input_unique_term_list)
    IDF = np.log10(len(input_document_list) / np.sum(OHE_array, axis = 0))
    TF = text_list_to_freq(input_document_list, input_unique_term_list)
    TF_IDF = TF * IDF
    return TF_IDF

text_list_to_TF_IDF(term_list, unique_term_list)

array([[0.47712125, 0.17609126, 0.17609126, 0.17609126, 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.17609126, 0.17609126, 0.17609126, 0.35218252,
        0.47712125, 0.47712125, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.17609126,
        0.        , 0.        , 0.47712125, 0.47712125]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [9]:
def cosine_sim_order(input_corpus, input_document_index):
    cosine_sim = np.zeros([len(input_corpus)])
    documents, unique_terms = get_corpus_lists(input_corpus)
    TF_IDF = text_list_to_TF_IDF(documents, unique_terms)
    TF_IDF_selected = TF_IDF[input_document_index]
    for document_index, document in enumerate(corpus):
        cosine_sim[document_index] = cosine_similarity(TF_IDF[document_index], TF_IDF_selected)
    sort = np.argsort(cosine_sim)

    return input_corpus[sort]

cosine_sim_order(corpus, 1)

array(['martes muchas gracias', 'que dia es hoy',
       'martes el dia de hoy es martes'], dtype='<U30')